In [1]:
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from sklearn.utils import resample,shuffle
import csv
import random
import math
import operator

In [2]:
def tfpn(testSet, predictions):
    tp = 0
    tn = 0
    fp = 0
    fn = 0
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][classcol] == predictions[x]:
            if(testSet[x][classcol] == 1):
                tp+=1
            else:
                tn+=1
        else:
            if(predictions[x] ==1):
                fp+=1
            else:
                fn+=1
    return (tp,tn,fp,fn)
def findclass(df):
    classs=None
    columns=df.columns.tolist()
    for i in range(len(columns)):
        if columns[i]=='class':
            classs=i
            break
    return classs
classcol=None
def loaddataset(file,perc,trainset=[],testset=[]):
    data=pd.read_csv(file)
    df_maj = data[data['class'] == 1]
    l = len(df_maj)
    #print(len(df_maj))
    df_min = data[data['class'] == 0]
    df_minor_up = resample(df_min,replace=True,n_samples=l,random_state=123)
    #print(len(df_minor_up))
    df_upsampled = pd.concat([df_maj,df_minor_up])
    df_upsampled=shuffle(df_upsampled)
    df=df_upsampled.drop(['spectrometric_redshift','Unnamed: 0', 'galex_objid', 'sdss_objid','pred'],1)
    global classcol 
    classcol=findclass(df)
    df=df[1:]
    dataset=list(df.values.tolist())
    length=len(dataset[0])
    count=0
    split=(len(dataset)-1)*perc
    for x in range(1,len(dataset)-1):
        for y in range(1,length):
            dataset[x][y]=float(dataset[x][y])
        if(count<split):
            trainset.append(dataset[x])
                #print(dataset[x])
        else:
            testset.append(dataset[x])
        count+=1
def euclideanDistance(instance1, instance2, length):
    distance = 0
    for x in range(length):
        distance += pow(float(instance1[x]) - float(instance2[x]), 2)
    return math.sqrt(distance)

def getNeighbors(trainingSet, testInstance, k):
    distances = []
    length = len(testInstance)
    for x in range(1,len(trainingSet)):
        dist = euclideanDistance(testInstance, trainingSet[x], length)
        distances.append((trainingSet[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors

def getResponse(neighbors):
    classVotes = {}
    for x in range(len(neighbors)):
        response = neighbors[x][classcol]
        if response in classVotes:
            classVotes[response] += 1
        else:
            classVotes[response] = 1
        sortedVotes = sorted(classVotes.items(), key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]

def getAccuracy(testSet, predictions):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][classcol] == predictions[x]:
            correct += 1
    print(len(testSet))
    return (correct/float(len(testSet))) * 100.0

In [4]:
from random import seed
from random import randrange
 
# Split a dataset into k folds
def cross_validation_split(dataset, folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / folds)
    for i in range(folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

In [9]:
# test cross validation split
def main(k):
    seed(1)
    trainingSet = []
    testset = []
    loaddataset('cat3.csv',0.8,trainingSet,testset)
    dataset = trainingSet+testset
    n_folds = 5
    folds = cross_validation_split(dataset, n_folds)
    predictions=[]
    test = []
    for i in range(n_folds):
        new_train = [item for item in dataset if item not in folds[i]]
        new_test = folds[i]
        print(i," training set")

        for x in range(len(new_test)):
            test.append(new_test[x])
            neighbors = getNeighbors(new_train, new_test[x], k)
            result = getResponse(neighbors)
            predictions.append(result)
            #print('>predicted=' + str(result) + ', actual=' + str(testSet[x][12]))
    l = tfpn(test,predictions)
    TP = l[0]
    TN = l[1]
    FP = l[2]
    FN = l[3]

    print("true positive :",TP)
    print("true negative :",TN)
    print("false positive :",FP)
    print("false negative :",FN)
    TPR = TP/(TP+FN)
    print( 'Sensitivity:{}'.format(TPR))
    TNR = TN/(TN+FP)
    print( 'Specificity:{}'.format(TNR))
    Precision = TP/(TP+FP)
    print( 'Precision:{}'.format(Precision))
    Recall = TP/(TP+FN)
    print( 'Recall:{}'.format(Recall))
    Acc = (TP+TN)/(TP+TN+FP+FN)
    print( 'Accuracy:{}'.format(Acc))
    Fscore = 2*(Precision*Recall)/(Precision+Recall)
    print( 'FScore:{}'.format(Fscore))

In [10]:
main(3)

0  training set
1  training set
2  training set
3  training set
4  training set
true positive : 3792
true negative : 2271
false positive : 1552
false negative : 30
Sensitivity:0.9921507064364207
Specificity:0.5940360973057808
Precision:0.7095808383233533
Recall:0.9921507064364207
Accuracy:0.7930673642903858
FScore:0.8274056295003274


In [11]:
main(7)

0  training set
1  training set
2  training set
3  training set
4  training set
true positive : 3766
true negative : 2661
false positive : 1163
false negative : 55
Sensitivity:0.9856058623397016
Specificity:0.6958682008368201
Precision:0.7640495029417732
Recall:0.9856058623397016
Accuracy:0.8406801831262263
FScore:0.8608


In [12]:
main(10)

0  training set
1  training set
2  training set
3  training set
4  training set
true positive : 3763
true negative : 2502
false positive : 1321
false negative : 59
Sensitivity:0.9845630559916274
Specificity:0.6544598482866858
Precision:0.7401652242328874
Recall:0.9845630559916274
Accuracy:0.8194898626553303
FScore:0.8450482820570401
